<a href="https://colab.research.google.com/github/AVICDEBBYH/MAP_CHECK/blob/main/HANA_CLOUD_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install generative-ai-hub-sdk
!pip install hdbcli
!pip install hana_ml
!pip install pandas
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.6/613.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.5/265.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.2:
      Successfully uninstalled pydantic_core-2.33.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 20.2 MB/s eta 0:00:00


In [2]:
import os
os.environ['AICORE_AUTH_URL'] = 'https://avic-subaccount-d2sdwrqf.authentication.jp10.hana.ondemand.com'
os.environ['AICORE_CLIENT_ID'] = 'sb-fca69612-5c8f-417b-b9a8-13cb08309fa7!b30307|aicore!b44'
os.environ['AICORE_RESOURCE_GROUP'] = 'default'
os.environ['AICORE_CLIENT_SECRET'] = 'c6a8afdb-a49b-44a7-a112-1067961e4cf7$B6NA7SnXbpcKL1Bt2myspoEI41ztf8duebAWd47WJrU='
os.environ['AICORE_BASE_URL'] = 'https://api.ai.prod.ap-northeast-1.aws.ml.hana.ondemand.com'
os.environ['HANA_VECTOR_USER'] = 'DBADMIN'
os.environ['HANA_VECTOR_PASS'] = 'Welcome1'
os.environ['HANA_HOST_VECTOR'] = 'dabb5e77-369b-4abb-a27c-0db6194e2418.hana.prod-jp10.hanacloud.ondemand.com'

In [3]:
"""In this scenario, we show how we could use genaihub SDK to retrieve embedded response from SAP HANA Cloud Vector.
   WE use two of the foundation models. Based on the use prompt, we use the "embedding model" to convert the user prompt
   to embedding. We use COSINE SIMILARITY to retrieve the closest embedding from HANA DB. Then the corresponding
   text is retrieved along with source file , actual text and Scoring.
   Then we provide the retrieved text as input to "falcon" model to determine the sentiment of the text.
    Please install the necessary packages
    pip install   generative-ai-hub-sdk ||hdbcli || hana_ml || python-dotenv || shapely
"""
#load all necessary packages
from gen_ai_hub.proxy.native.openai import embeddings
from gen_ai_hub.proxy.native.openai import completions
from gen_ai_hub.proxy.native.openai import chat
from  hdbcli import dbapi
import pandas as pd
import hana_ml.dataframe as dataframe
import os
from dotenv import load_dotenv
load_dotenv()
# provide the credentials to connect to HANA Cloud DB
HANA_HOST = os.getenv('HANA_HOST_VECTOR')
HANA_USER = os.getenv('HANA_VECTOR_USER')
HANA_PASSWD = os.getenv('HANA_VECTOR_PASS')
# Establish connections
conn = dataframe.ConnectionContext(
    address= HANA_HOST,
    port=443,
    user=HANA_USER,
    password=HANA_PASSWD,
    encrypt='true'
)
conn1 = dbapi.connect(
    address=HANA_HOST,
    port=443,
    user=HANA_USER,
    password=HANA_PASSWD
)
#Here is the User Prompt. Change it  to query against the text that was ingested
prompt = "請問公司加班規定有哪些？"
#Here we convert your input to vector using the Azure ada embedding model
res = embeddings.create(input=prompt, model="text-embedding-ada-002")
query_vector = res.data[0].embedding
# we query the DB with the embedded response from user prompt
sql = '''SELECT TOP {k}  "FILENAME","CONTENT" , TO_NVARCHAR("EMBEDDING") AS VECTOR_STR ,"{metric}"("EMBEDDING", TO_REAL_VECTOR('{qv}')) as SCORING
                  FROM "VECTOR_DEMO"."HR_POLICY_RAG"
                  ORDER BY "{metric}"("EMBEDDING", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=3, metric="COSINE_SIMILARITY", qv=query_vector, sort="DESC")
hdf = conn.sql(sql)
res = hdf.head(10).collect()
#collect the response from previous SQL
#NOw loop around every row to get filename, text and data
if not res.empty:
    db_results = [(row['FILENAME'],row['CONTENT'], row['SCORING']) for _, row in res.iterrows()]
    new_results = []
    for i in range(len(db_results)):
        if i < len(db_results):
            filename, text,  scoring = db_results[i]
            #we are  getting the sentiment for every text using the below prompt
            sentiment_prompt = f"Provide sentiment in exactly one word for the: '{text}'"
            #in order to get the prompt we call the falcon model from aicore config
            # sentiment_output = completions.create(model_name="gpt-35-turbo", prompt=sentiment_prompt, max_tokens=60)
            # sentiment = sentiment_output.choices[0].text.strip()

            messages = [ {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": sentiment_prompt} ]

            print(sentiment_prompt)

            kwargs = dict(model_name='gpt-35-turbo', messages=messages)
            response = chat.completions.create(**kwargs)
            # sentiment = response['choices'][0]['message']['content'].strip()
            sentiment = response.choices[0].message.content.strip()

            print(sentiment)

            new_tuple =  (filename,text, scoring, sentiment)
            new_results.append(new_tuple)
#transform the results to DF and check the output
df_new_results = pd.DataFrame(new_results)
print(df_new_results)

Provide sentiment in exactly one word for the: '第三十八條：本公司因業務需求，員工有於正常工作時間以外工作之必要
者，由單位主管與員工商議後：
一、得將本規則第三十四條所定之工作時間延長之。
二、得於休假日工作。'
Necessary
Provide sentiment in exactly one word for the: '第三十五條：前條所訂工作時間，經員工半數以上同意得做下列變更：
一、四週內正常工作時數以每日不得超過二小時方式，分配
於其他工作日。
第 5 頁
--- 頁9 ---
二、當日正常工時達十小時者，其延長之工作時間不得超過
二小時。
三、兩週內應有二日之休息作為例假。
四、女性員工得於夜間工作，但雇主應提供完善安全衛生設
施。'
Neutral
Provide sentiment in exactly one word for the: '第三十四條：加班費之計算，優於勞基法以月薪除上22天為日薪，其他倍
率計算以勞工局規範為準。
第 五 章 工 作 時 間 、 休 息 、 休 假 、 請 假
第 一 節 工 作 時 間'
Neutral
                    0                                                  1  \
0  艾威科人事管理規則 2022.pdf  第三十八條：本公司因業務需求，員工有於正常工作時間以外工作之必要\n者，由單位主管與員工商議...   
1  艾威科人事管理規則 2022.pdf  第三十五條：前條所訂工作時間，經員工半數以上同意得做下列變更：\n一、四週內正常工作時數以每...   
2  艾威科人事管理規則 2022.pdf  第三十四條：加班費之計算，優於勞基法以月薪除上22天為日薪，其他倍\n率計算以勞工局規範為準...   

          2          3  
0  0.878344  Necessary  
1  0.870861    Neutral  
2  0.867536    Neutral  
